# Proyecto 3

**Tema:** Resultados Saber 11


|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |


## Entrenamiento Modelo

## Librerías Requeridas

In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Carga de Datos

In [2]:
df = pd.read_csv("../../../Data/Transformed/Discrete.csv")

In [3]:
df.drop(['colegio_calendario', 'colegio_jornada', 'colegio_genero', 'estudiante_genero'], axis=1, inplace=True)

In [4]:
df.head()

,colegio_rural,colegio_bilingue,colegio_privado,familia_estrato,madre_educacion,padre_educacion,computador,internet,puntaje,colegio_mcpio_distinto,presentacion_mcpio_distinto
0,0,0,0,1,4,2,1,1,5,0,0
1,0,0,0,1,3,3,0,0,4,0,1
2,0,0,0,1,3,3,0,0,4,0,1
3,1,0,1,4,9,10,1,1,7,0,0
4,0,0,0,1,2,2,1,1,5,0,0


In [5]:
df.shape

(3295965, 11)

## Train Test Sets

In [6]:
variables = [column for column in df.columns if column != 'puntaje']

In [7]:
X = df[variables].values

y = df['puntaje'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
df_train = pd.DataFrame(X_train, columns=variables)
df_train['puntaje'] = y_train

df_test = pd.DataFrame(X_test, columns=variables)
df_test['puntaje'] = y_test

In [10]:
df_train.shape, df_test.shape

((2471973, 11), (823992, 11))

## Test

In [10]:
def trainModel(df_train):
    model = BayesianNetwork(
        [
            # Colegio Rural
            ('colegio_rural', 'presentacion_mcpio_distinto'),
            ('colegio_rural', 'colegio_privado'),
            ('colegio_rural', 'colegio_mcpio_distinto'),
            
            # Colegio Bilingue
            ('colegio_bilingue', 'puntaje'),
            
            # Colegio Privado
            ('colegio_privado', 'presentacion_mcpio_distinto'),
            ('colegio_privado', 'colegio_bilingue'),
            ('colegio_privado', 'puntaje'),
            
            # Estrato
            ('familia_estrato', 'colegio_privado'),
            ('familia_estrato', 'internet'),
            ('familia_estrato', 'computador'),
            ('familia_estrato', 'puntaje'),
            
            # Madre Educacion
            ('madre_educacion', 'familia_estrato'),
            
            # Padre Educacion
            ('padre_educacion', 'familia_estrato'),
            
            # Computador
            ('computador', 'puntaje'),
            
            # Internet
            ('internet', 'puntaje'),
                    
            # Presenta otro Municipio
            ('presentacion_mcpio_distinto', 'puntaje'),
        ]
    )
    model.fit (
        data=df_train,
        estimator=MaximumLikelihoodEstimator
    )
    model.check_model()
    
    return model

In [11]:
def getEvidence(nodes, row):
    
    evidence = {}
    for node in nodes:
        if node != 'puntaje':
            evidence[node] = (row[node])
    
    return evidence

In [23]:
def inferenceEvidence(evidence, infer):
    
    prob = infer.query(variables=['puntaje'], evidence=evidence)
    
    return prob.values.tolist()

In [24]:
def getClassification(probs):
    maxi = 0
    clase = 0
    for i in range(0, 10):
        if probs[i] > maxi:
            maxi = probs[i]
            clase = i+1
    
    return clase

In [14]:
model = trainModel(df_train)

In [15]:
print(model)

BayesianNetwork with 11 nodes and 16 edges


In [28]:
nodes = model.nodes

infer = VariableElimination(model)

test = []
for i in range(0, len(df_test)):
    row = df_test.iloc[i]
    evidence = getEvidence(nodes=nodes, row=row)
    probs = inferenceEvidence(evidence, infer)
    classification = getClassification(probs)
    test.append(classification)    

df_test['test'] = test

In [ ]:
df_test

,colegio_rural,colegio_bilingue,colegio_privado,familia_estrato,madre_educacion,padre_educacion,computador,internet,colegio_mcpio_distinto,presentacion_mcpio_distinto,puntaje,test
0,0,0,0,1,5,5,0,0,0,1,5,5
1,0,0,0,3,4,3,1,1,1,1,6,6
2,0,0,1,4,9,9,1,1,0,0,7,7
3,0,0,0,2,4,4,0,0,0,0,5,5
4,0,0,0,3,7,9,1,1,0,0,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...
823987,0,0,0,1,2,2,1,0,0,0,6,5
823988,0,0,1,2,8,4,1,1,0,0,6,6
823989,0,0,0,2,5,5,1,1,0,0,5,6
823990,0,0,0,2,10,4,1,1,0,0,4,6


## Guardar Datos

In [ ]:
df_test.to_csv('../../../Data/Transformed/Test.csv', index=False)